In [ ]:
!pip install -U -q transformers accelerate bitsandbytes human_eval

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from typing import List
import torch

model_name = "/mnt/workspace/model/qwen05bChat"
max_new_tokens = 1024
device = "cuda"
model_max_length = 1024

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=model_max_length)


def generate(prompts: List[str]) -> List[str]:
    global sample_number
    sample_number += 1
    print("sample_number:", sample_number)
    print(len(prompts))
    for prompt in prompts:
        print("input_prompt:", prompt)
    model_inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    print("model_inputs type:", type(model_inputs))
    #for modle_input in model_inputs:
        #print("input_tokens shape:", len(modle_input))
    with torch.no_grad():
        generated_ids = model.generate(**model_inputs, max_new_tokens=max_new_tokens, do_sample=False)
        print("generated_ids shape:",generated_ids.shape)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [7]:
import time
texts = ["Hello, my name is", ""]
a = time.time()
print((generate(texts)))
b = time.time()
c = b - a
print(c)

3.0025339126586914


In [ ]:
from human_eval.data import write_jsonl, read_problems

sample_number = 0
num_samples_per_task = 1

problems = read_problems()
#print(problems)
samples = [
    dict(task_id=task_id, completion=generate([problems[task_id]["prompt"]]))
    #for task_id in list(problems.keys())[:10]
    for task_id in problems
    for _ in range(num_samples_per_task)
]
print("sample size:", len(samples))
write_jsonl("samples3.jsonl", samples)